In [1]:
import pandas as pd
import os
import json
import sys
import sqlite3
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
from calculate import Scorer

In [2]:
df = pd.read_csv('../data/scores.csv')
with open('../data/weeks.json', 'r') as f:
    weeks = json.loads(f.read())

def get_week(row):
    week = row['old_week']
    new_week = weeks[week] if week in weeks else week
    return week if week == 'Super Bowl' else week.split(maxsplit=1)[1]

def get_week_order(row):
    week = row['week']
    last = week.split()[1]
    if last.isdigit():
        return int(last)
    if week == 'Wild Card Round':
        return 20
    if week == 'Divisional Round':
        return 21
    if week == 'Conf Champ' or week == 'Championship Round':
        return 22
    if week == 'Super Bowl':
        return 23
    return 0
    

def get_year(row):
    week = row['old_week']
    if week == 'Super Bowl':
        return int(row['date'].split(',')[1].strip()) - 1
    return int(week.split()[0])

def get_quarter(row):
    quarter = row['quarter']
    if quarter.replace('.', '').isdigit():
        return int(float(quarter))
    # Set all OT as 5th quarter
    return 5

df = df.set_index('Unnamed: 0', drop=False)
df = df.rename(columns={'week': 'old_week', 'Tm': 'scoring_team', 'Quarter': 'quarter', 'Detail': 'detail'})
df['week'] = df.apply(get_week, axis=1)
df['year'] = df.apply(get_year, axis=1)
df['week_order'] = df.apply(get_week_order, axis=1)
df['quarter'] = df.apply(get_quarter, axis=1)
#df['quarter'] = df['quarter'].astype(float).astype(int)#df.apply(lambda row: int(row['quarter']), axis=1)
#df['week_order'] = df['week_order'].astype(int)

df = df.drop(columns=['old_week'])
df[(df['week'] == 'Super Bowl')]

,Unnamed: 0,quarter,Time,scoring_team,detail,team1_score,team2_score,date,team1_city,team2_city,team1_mascot,team2_mascot,team1_original_city,team2_original_city,team1_original_mascot,team2_original_mascot,week,year,week_order
Unnamed: 0,,,,,,,,,,,,,,,,,,,
2039,2039,1,14:46,Bears,Devin Hester 92 yard kickoff return (Robbie Go...,0,7,"Feb 4, 2007",Indianapolis,Chicago,Colts,Bears,Indianapolis,Chicago,Colts,Bears,Super Bowl,2006,23
2040,2040,1,6:50,Colts,Reggie Wayne 53 yard pass from Peyton Manning ...,6,7,"Feb 4, 2007",Indianapolis,Chicago,Colts,Bears,Indianapolis,Chicago,Colts,Bears,Super Bowl,2006,23
2041,2041,1,4:34,Bears,Muhsin Muhammad 4 yard pass from Rex Grossman ...,6,14,"Feb 4, 2007",Indianapolis,Chicago,Colts,Bears,Indianapolis,Chicago,Colts,Bears,Super Bowl,2006,23
2042,2042,2,11:17,Colts,Adam Vinatieri 29 yard field goal,9,14,"Feb 4, 2007",Indianapolis,Chicago,Colts,Bears,Indianapolis,Chicago,Colts,Bears,Super Bowl,2006,23
2043,2043,2,6:09,Colts,Dominic Rhodes 1 yard rush (Adam Vinatieri kick),16,14,"Feb 4, 2007",Indianapolis,Chicago,Colts,Bears,Indianapolis,Chicago,Colts,Bears,Super Bowl,2006,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112681,112681,3,2:06,Patriots,James White 5 yard pass from Tom Brady (Stephe...,9,28,"Feb 5, 2017",New England,Atlanta,Patriots,Falcons,New England,Atlanta,Patriots,Falcons,Super Bowl,2016,23
112682,112682,4,9:44,Patriots,Stephen Gostkowski 33 yard field goal,12,28,"Feb 5, 2017",New England,Atlanta,Patriots,Falcons,New England,Atlanta,Patriots,Falcons,Super Bowl,2016,23
112683,112683,4,5:56,Patriots,Danny Amendola 6 yard pass from Tom Brady (Jam...,20,28,"Feb 5, 2017",New England,Atlanta,Patriots,Falcons,New England,Atlanta,Patriots,Falcons,Super Bowl,2016,23


In [3]:

for group_name, df_group in df.groupby(['date', 'team1_city', 'team2_city']):
    rows = df_group.iterrows()
    length = len(list(rows))
    scorer1 = Scorer()
    scorer2 = Scorer()
    
    for row_index, row in df_group.iterrows():
        score1 = scorer1.calculate(float(row['team1_score']), float(row['team2_score']), row['quarter'], row_index == length - 1)
        score2 = scorer2.calculate(float(row['team2_score']), float(row['team1_score']), row['quarter'], row_index == length - 1)
        df.at[row['Unnamed: 0'], 'score1'] = score1
        df.at[row['Unnamed: 0'], 'score2'] = score2
df


,Unnamed: 0,quarter,Time,scoring_team,detail,team1_score,team2_score,date,team1_city,team2_city,...,team2_mascot,team1_original_city,team2_original_city,team1_original_mascot,team2_original_mascot,week,year,week_order,score1,score2
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,0,1,6:05,Dolphins,Olindo Mare 37 yard field goal,0,3,"Oct 24, 1999",Philadelphia,Miami,...,Dolphins,Philadelphia,Miami,Eagles,Dolphins,Week 7,1999,7,-14.599862,14.599862
1,1,1,1:04,Dolphins,J.J. Johnson 18 yard rush (Olindo Mare kick),0,10,"Oct 24, 1999",Philadelphia,Miami,...,Dolphins,Philadelphia,Miami,Eagles,Dolphins,Week 7,1999,7,-36.926499,36.926499
2,2,2,11:10,Eagles,David Akers 53 yard field goal,3,10,"Oct 24, 1999",Philadelphia,Miami,...,Dolphins,Philadelphia,Miami,Eagles,Dolphins,Week 7,1999,7,-36.152985,36.152985
3,3,2,6:26,Dolphins,Olindo Mare 37 yard field goal,3,13,"Oct 24, 1999",Philadelphia,Miami,...,Dolphins,Philadelphia,Miami,Eagles,Dolphins,Week 7,1999,7,-50.553671,50.553671
4,4,3,3:47,Eagles,Brian Dawkins 67 yard interception return (Nor...,10,13,"Oct 24, 1999",Philadelphia,Miami,...,Dolphins,Philadelphia,Miami,Eagles,Dolphins,Week 7,1999,7,-34.973128,34.973128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113569,113569,3,NaN,Cowboys,Daryl Johnston 1 yard rush (Eddie Murray kick),27,6,"Dec 12, 1993",Dallas,Minnesota,...,Vikings,Dallas,Minnesota,Cowboys,Vikings,Week 15,1993,15,148.828879,-148.828879
113570,113570,3,NaN,Vikings,Scottie Graham 1 yard rush (Fuad Reveiz kick),27,13,"Dec 12, 1993",Dallas,Minnesota,...,Vikings,Dallas,Minnesota,Cowboys,Vikings,Week 15,1993,15,133.582293,-133.582293
113571,113571,4,NaN,Cowboys,Eddie Murray 46 yard field goal,30,13,"Dec 12, 1993",Dallas,Minnesota,...,Vikings,Dallas,Minnesota,Cowboys,Vikings,Week 15,1993,15,108.055555,-108.055555


In [4]:
#df.to_csv('../data/scores_with_index.csv')
if os.path.exists('test.db'):
    os.remove('test.db')
conn = sqlite3.connect('test.db')
c = conn.cursor()
df.to_sql('scores_temp', conn, index=False, if_exists='replace')
c.execute('''
CREATE TABLE IF NOT EXISTS weeks (
    week_id INTEGER PRIMARY KEY,
    year INTEGER,
    week TEXT,
    week_order INTEGER
)''')

c.execute('''
CREATE TABLE IF NOT EXISTS teams (
    team_id INTEGER PRIMARY KEY,
    original_city TEXT,
    original_mascot TEXT,
    city TEXT,
    mascot TEXT
)''')

c.execute('''
CREATE TABLE IF NOT EXISTS games (
    game_id INTEGER PRIMARY KEY,
    team1_id INTEGER,
    team2_id INTEGER,
    week_id INTEGER
)''')

c.execute('''
CREATE TABLE IF NOT EXISTS scores (
    score_id INTEGER PRIMARY KEY,
    game_id INTEGER,
    quarter INTEGER,
    time TEXT,
    scoring_team_id INTEGER,
    detail TEXT,
    team1_game_score INTEGER,
    team2_game_score INTEGER,
    team1_misery_index INTEGER,
    team2_misery_index INTEGER,
    score_order INTEGER
)''')


OperationalError: near "order": syntax error